# Loop 8 Strategic Analysis

## Key Findings from Research

1. **Top kernels use C++ optimizers (bbox3)** - These are pre-compiled binaries that run simulated annealing
2. **The baseline is already at a local optimum** - Fractional translation found ZERO improvements
3. **Gap to target: 1.73 points** (70.615 → 68.886)
4. **Top teams achieve ~68.5** - This proves the gap IS closable

## What's NOT Working
- Ensemble approaches (3 failures due to overlaps)
- Fractional translation (baseline is already optimized)
- Running same optimizer with different parameters

## What COULD Work
1. **Implement SA from scratch in Python** (not using binaries)
2. **Focus on small N values** (N=2-20 have largest theoretical gaps)
3. **Use Numba/JIT for speed**
4. **Rotation optimization** (not just translation)

In [ ]:
# Analyze the theoretical gap for small N values
import json
import numpy as np

# Load baseline scores
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

# The baseline score is 70.615107
baseline_total = 70.615107
target = 68.885544
gap = baseline_total - target

print(f"Current baseline: {baseline_total:.6f}")
print(f"Target: {target:.6f}")
print(f"Gap to close: {gap:.6f}")
print(f"Gap percentage: {100*gap/baseline_total:.2f}%")

In [ ]:
# Analyze per-N contributions to understand where improvements are possible
# Load the baseline submission
import pandas as pd
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

VALID_BASELINE = '/home/nonroot/snapshots/santa-2025/21337353543/submission/submission.csv'

TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def create_tree(x, y, angle):
    poly = Polygon(zip(TX, TY))
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    poly = affinity.translate(poly, x, y)
    return poly

def calculate_side(trees):
    polys = [create_tree(*t) for t in trees]
    union = unary_union(polys)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def load_baseline(path):
    trees_by_n = {}
    with open(path, 'r') as f:
        next(f)  # Skip header
        for line in f:
            parts = line.strip().split(',')
            if len(parts) != 4:
                continue
            id_val, x_str, y_str, deg_str = parts
            n = int(id_val.split('_')[0])
            x = float(x_str[1:] if x_str.startswith('s') else x_str)
            y = float(y_str[1:] if y_str.startswith('s') else y_str)
            angle = float(deg_str[1:] if deg_str.startswith('s') else deg_str)
            if n not in trees_by_n:
                trees_by_n[n] = []
            trees_by_n[n].append((x, y, angle))
    return trees_by_n

baseline_trees = load_baseline(VALID_BASELINE)
print(f"Loaded {len(baseline_trees)} N values")

In [ ]:
# Calculate per-N scores and identify where improvements are most valuable
per_n_scores = {}
for n in range(1, 201):
    side = calculate_side(baseline_trees[n])
    score = (side ** 2) / n
    per_n_scores[n] = {'side': side, 'score': score}

# Sort by score contribution (highest first)
sorted_by_score = sorted(per_n_scores.items(), key=lambda x: x[1]['score'], reverse=True)

print("Top 20 N values by score contribution:")
print("="*50)
for n, data in sorted_by_score[:20]:
    print(f"N={n:3d}: side={data['side']:.6f}, score={data['score']:.6f}")

print(f"\nTotal score: {sum(d['score'] for d in per_n_scores.values()):.6f}")

In [ ]:
# Calculate theoretical minimum for small N
# For N=1, the optimal is a single tree at 45 degrees (already known)
# For N=2+, we need to estimate based on tree geometry

# Tree bounding box at optimal rotation (45 degrees)
import math

def get_tree_bbox_at_angle(angle):
    """Get bounding box dimensions for a tree at given angle."""
    poly = Polygon(zip(TX, TY))
    poly = affinity.rotate(poly, angle, origin=(0, 0))
    bounds = poly.bounds
    return bounds[2] - bounds[0], bounds[3] - bounds[1]  # width, height

# Find optimal angle for single tree
best_angle = 0
best_max_dim = float('inf')
for angle in range(0, 360):
    w, h = get_tree_bbox_at_angle(angle)
    max_dim = max(w, h)
    if max_dim < best_max_dim:
        best_max_dim = max_dim
        best_angle = angle

print(f"Optimal single tree angle: {best_angle}°")
print(f"Optimal single tree side: {best_max_dim:.6f}")
print(f"Optimal N=1 score: {best_max_dim**2:.6f}")
print(f"Baseline N=1 score: {per_n_scores[1]['score']:.6f}")

In [ ]:
# Analyze the gap for small N values
# Theoretical minimum assumes perfect packing (which may not be achievable)

print("\nAnalysis of small N values:")
print("="*60)
print(f"{'N':>3} | {'Baseline':>10} | {'Side':>8} | {'Trees/Area':>10}")
print("-"*60)

for n in range(1, 21):
    score = per_n_scores[n]['score']
    side = per_n_scores[n]['side']
    area = side ** 2
    trees_per_area = n / area
    print(f"{n:3d} | {score:10.6f} | {side:8.4f} | {trees_per_area:10.4f}")

print(f"\nSum of N=1-20 scores: {sum(per_n_scores[n]['score'] for n in range(1, 21)):.6f}")

In [ ]:
# Key insight: The baseline is already highly optimized
# To improve, we need to either:
# 1. Find better configurations for specific N values (hard - already optimized)
# 2. Use a fundamentally different approach (e.g., simulated annealing from scratch)

# Let's check what the top kernels achieve
# From research: top kernels achieve ~68.5, we're at 70.6
# That's a 2.1 point gap

# The gap is distributed across all N values
# If we could improve each N by 1%, we'd get:
improvement_1pct = sum(per_n_scores[n]['score'] * 0.01 for n in range(1, 201))
print(f"1% improvement across all N: {improvement_1pct:.6f}")

# To close the 1.73 gap, we need:
required_pct = 1.73 / sum(per_n_scores[n]['score'] for n in range(1, 201)) * 100
print(f"Required improvement: {required_pct:.2f}%")

# This is achievable with better optimization algorithms!

In [ ]:
# Strategy: Implement simulated annealing from scratch
# Key components:
# 1. Random perturbation (translation + rotation)
# 2. Temperature schedule
# 3. Accept worse solutions with probability exp(-delta/T)
# 4. Focus on small N first (faster to test)

print("\n" + "="*60)
print("RECOMMENDED STRATEGY FOR NEXT EXPERIMENT")
print("="*60)
print("""
1. IMPLEMENT SIMULATED ANNEALING FROM SCRATCH (no binaries!)
   - Start with small N (N=10, N=20, N=30)
   - Use Numba for speed
   - Temperature schedule: T_start=1.0, T_end=0.00001, cooling=0.999
   - Perturbations: dx, dy in [-0.1, 0.1], da in [-5, 5] degrees

2. KEY INSIGHT: The baseline is at a LOCAL optimum
   - Fractional translation can't escape it
   - SA with temperature can escape local optima
   - Need to accept worse solutions temporarily

3. FOCUS ON HIGH-IMPACT N VALUES:
   - N=1: Already optimal (45°)
   - N=2-10: Highest score contribution per tree
   - N=11-50: Medium impact
   - N=51-200: Lower impact but many values

4. VALIDATION:
   - Use integer scaling (1e18) for overlap detection
   - Match Kaggle's precision requirements
""")

print(f"\nTarget: {target:.6f}")
print(f"Current: {baseline_total:.6f}")
print(f"Gap: {gap:.6f} ({required_pct:.2f}% improvement needed)")